# CH.12_Advanced_pandas

## 0. Import Packages

In [1]:
import numpy as np
import pandas as pd

## 1. Categorical Data

In [3]:
values = pd.Series(['apple', 'orange', 'apple',
                   'apple'] * 2)

print(values)
print(pd.unique(values))
print(pd.value_counts(values))

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object
['apple' 'orange']
apple     6
orange    2
dtype: int64


In [6]:
values = pd.Series([0, 1, 0, 0] * 2)
dim = pd.Series(['apple', 'orange'])

print(values)
print(dim)
print(dim.take(values))

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64
0     apple
1    orange
dtype: object
0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object


In [8]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2
N = len(fruits)
df = pd.DataFrame({'fruit': fruits,
                  'basket_id': np.arange(N),
                  'count': np.random.randint(3, 15, size=N),
                  'weight': np.random.uniform(0, 4, size=N)},
                 columns=['basket_id', 'fruit', 'count', 'weight'])
df

,basket_id,fruit,count,weight
0,0,apple,10,1.601744
1,1,orange,7,0.421963
2,2,apple,14,0.188631
3,3,apple,6,3.164012
4,4,apple,11,2.232885
5,5,orange,9,0.175249
6,6,apple,14,2.053393
7,7,apple,13,3.311375


In [11]:
fruit_cat = df['fruit'].astype('category')

print(fruit_cat)

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]


In [14]:
c = fruit_cat.values

print(type(c))
print(c.categories)
print(c.codes)

<class 'pandas.core.arrays.categorical.Categorical'>
Index(['apple', 'orange'], dtype='object')
[0 1 0 0 0 1 0 0]


In [15]:
df['fruit'] = df['fruit'].astype('category')
df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [16]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]

my_cats_2 = pd.Categorical.from_codes(codes, categories)
ordered_cat = pd.Categorical.from_codes(codes, categories,
                                       ordered=True)
my_cats_2.as_ordered()

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

In [17]:
draws = np.random.randn(1000)
bins = pd.qcut(draws, 4)
bins

[(-3.487, -0.693], (-3.487, -0.693], (-3.487, -0.693], (0.627, 3.442], (-0.693, -0.0325], ..., (-3.487, -0.693], (0.627, 3.442], (-0.0325, 0.627], (-3.487, -0.693], (-0.693, -0.0325]]
Length: 1000
Categories (4, interval[float64]): [(-3.487, -0.693] < (-0.693, -0.0325] < (-0.0325, 0.627] < (0.627, 3.442]]

In [18]:
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
bins.codes[:10]

array([0, 0, 0, 3, 1, 3, 0, 0, 1, 3], dtype=int8)

In [19]:
bins = pd.Series(bins, name='quartile')
results = (pd.Series(draws)
          .groupby(bins)
          .agg(['count', 'min', 'max'])
          .reset_index())
results

,quartile,count,min,max
0,Q1,250,-3.485820,-0.695769
1,Q2,250,-0.692061,-0.032516
2,Q3,250,-0.032417,0.626580
3,Q4,250,0.628364,3.442321


## 2. Computation with categories

In [20]:
N = 10000000
draws = pd.Series(np.random.randn(N))
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))
categories = labels.astype('category')

In [21]:
labels.memory_usage()

80000128

In [22]:
categories.memory_usage()

10000320

In [23]:
%time _ = labels.astype('category')

Wall time: 446 ms


## 3. Categorical methods

In [25]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)
cat_s = s.astype('category')

print(cat_s)
print(cat_s.cat.codes)
print(cat_s.cat.categories)

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]
0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8
Index(['a', 'b', 'c', 'd'], dtype='object')


In [27]:
actual_categories = ['a', 'b', 'c', 'd', 'e']
cat_s2 = cat_s.cat.set_categories(actual_categories)

print(cat_s2)
print(cat_s.value_counts())
print(cat_s2.value_counts())

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): [a, b, c, d, e]
d    2
c    2
b    2
a    2
dtype: int64
d    2
c    2
b    2
a    2
e    0
dtype: int64


In [28]:
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]

print(cat_s3)
print(cat_s3.cat.remove_unused_categories())

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): [a, b, c, d]
0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): [a, b]


## 4. Advanced groupby use

In [33]:
df = pd.DataFrame({'key': ['a','b','c'] * 4,
                   'value': np.arange(12.)})
g = df.groupby('key').value

print('[1]\n', g.mean())
print('[2]\n', g.transform(lambda x: x.mean()))
print('[3]\n', g.transform('mean'))
print('[4]\n', g.transform(lambda x: x * 2))
print('[5]\n', g.transform(lambda x: x.rank(ascending=False)))

[1]
 key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64
[2]
 0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64
[3]
 0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64
[4]
 0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64
[5]
 0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64


In [34]:
def normalize(x):
    return (x - x.mean()) / x.std()

print(g.transform(normalize))
print(g.apply(normalize))
print(g.transform('mean'))

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64
0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64
0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64


In [36]:
normalized = (df['value'] - g.transform('mean')) / g.transform('std')
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [37]:
N = 15
times = pd.date_range('2017-05-20 00:00', freq='1min', periods=N)
df = pd.DataFrame({'time': times,
                   'value': np.arange(N)})

df.set_index('time').resample('5min').count()
df2 = pd.DataFrame({'time': times.repeat(3),
                    'key': np.tile(['a', 'b', 'c'], N),
                    'value': np.arange(N * 3.)})
print(df2[:7])

                 time key  value
0 2017-05-20 00:00:00   a    0.0
1 2017-05-20 00:00:00   b    1.0
2 2017-05-20 00:00:00   c    2.0
3 2017-05-20 00:01:00   a    3.0
4 2017-05-20 00:01:00   b    4.0
5 2017-05-20 00:01:00   c    5.0
6 2017-05-20 00:02:00   a    6.0
